In [ ]:
import pandas as pd
import numpy as np
import io
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import re

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.compose import make_column_transformer
from sklearn.metrics import f1_score
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

slo_re = pd.read_excel('excel_file_path')
slo_re.tail()

In [ ]:
# Convert Lot size and Square footage to float

def convert_lot_size(lot_size):
    # Check if the entry is blank or NaN
    if pd.isna(lot_size) or lot_size.strip() == "":
        return np.nan

    # Remove commas
    lot_size = lot_size.replace(',', '')

    if 'sqft' in lot_size:
        # Remove 'sqft' and convert to int
        return int(float(lot_size.replace(' sqft', '')))
    elif 'acres' in lot_size:
        # Remove 'acres', convert to float, then to square feet, and finally to int
        acres = float(lot_size.replace(' acres', ''))
        sqft = acres * 43560
        return int(sqft)
    else:
        raise ValueError("Unexpected lot size format")

# Apply the function
slo_re['Lot Size'] = slo_re['Lot Size'].apply(convert_lot_size)
slo_re['Square Footage'] = slo_re['Square Footage'].apply(convert_lot_size)
slo_re.head()

,MLS #,List Price,Sell Price,Address,Structure Type,Property Type,Beds,Full Baths,Half Baths,Square Footage,...,Closed On,Stories,Style,Attached Garage,Road Frontage,New Construction,Year Built,Common Interest,Days On Market,District
0,MLS #SC24127762,663500,663500,"570 Peach Street #23 San Luis Obispo, CA 93401...",House,Residential,2,2.0,0,918,...,2024-07-26,1,NaN,NaN,NaN,No,1975,Condominium,4.0,downtown
1,MLS #SC24091640,2195000,2000000,"570 Pacific San Luis Obispo, CA 93401",House,Residential,3,3.0,1,2227,...,2024-07-25,3,NaN,Yes,NaN,No,2024,NaN,57.0,downtown
2,MLS #SC24073717,1225000,1195000,"771 Toro Street San Luis Obispo, CA 93401-2830",House,Residential,2,2.0,0,1200,...,2024-07-19,1,NaN,NaN,NaN,No,1925,NaN,68.0,downtown
3,MLS #NS24109649,729000,729000,"1739 Johnson Avenue San Luis Obispo, CA 93401-...",House,Residential,2,1.0,0,900,...,2024-07-11,1,NaN,No,City Street,No,1925,NaN,8.0,downtown
4,MLS #SC23208408,1194000,1300000,"1143 Peach Street San Luis Obispo, CA 93401-2818",House,Residential,2,2.0,0,1100,...,2024-06-24,1,NaN,NaN,City Street,No,1906,NaN,143.0,downtown


In [ ]:
# Convert HOA Fee to float

def convert_hoa_fee(hoa_fee):

    if isinstance(hoa_fee, (int, float)):
        return float(hoa_fee)

    hoa_fee = str(hoa_fee).strip().replace('$', '').replace(',', '')

    if hoa_fee in ["", "0"]:
        return np.nan

    if 'Monthly' in hoa_fee:
        hoa_fee = hoa_fee.replace(' Monthly', '')
        try:
            return float(hoa_fee)
        except ValueError:
            print(f"Failed to convert HOA fee: {hoa_fee}")
            return np.nan

    elif 'Annually' in hoa_fee:
        hoa_fee = hoa_fee.replace(' Annually', '')
        try:
            return float(hoa_fee) / 12
        except ValueError:
            print(f"Failed to convert HOA fee: {hoa_fee}")
            return np.nan

    print(f"Unexpected HOA fee format: {hoa_fee}")
    return np.nan

slo_re['HOA Fee'] = slo_re['HOA Fee'].apply(convert_hoa_fee)

slo_re.head()

,MLS #,List Price,Sell Price,Address,Structure Type,Property Type,Beds,Full Baths,Half Baths,Square Footage,...,Closed On,Stories,Style,Attached Garage,Road Frontage,New Construction,Year Built,Common Interest,Days On Market,District
0,MLS #SC24127762,663500,663500,"570 Peach Street #23 San Luis Obispo, CA 93401...",House,Residential,2,2.0,0,918,...,2024-07-26,1,NaN,NaN,NaN,No,1975,Condominium,4.0,downtown
1,MLS #SC24091640,2195000,2000000,"570 Pacific San Luis Obispo, CA 93401",House,Residential,3,3.0,1,2227,...,2024-07-25,3,NaN,Yes,NaN,No,2024,NaN,57.0,downtown
2,MLS #SC24073717,1225000,1195000,"771 Toro Street San Luis Obispo, CA 93401-2830",House,Residential,2,2.0,0,1200,...,2024-07-19,1,NaN,NaN,NaN,No,1925,NaN,68.0,downtown
3,MLS #NS24109649,729000,729000,"1739 Johnson Avenue San Luis Obispo, CA 93401-...",House,Residential,2,1.0,0,900,...,2024-07-11,1,NaN,No,City Street,No,1925,NaN,8.0,downtown
4,MLS #SC23208408,1194000,1300000,"1143 Peach Street San Luis Obispo, CA 93401-2818",House,Residential,2,2.0,0,1100,...,2024-06-24,1,NaN,NaN,City Street,No,1906,NaN,143.0,downtown


In [ ]:
slo_re['Lot Size'].unique()

array([8.7100000e+02, 2.4990000e+03, 4.4000000e+03, 4.0000000e+03,
       3.8000000e+03, 5.5000000e+03, 1.4630000e+03, 8.7800000e+02,
       7.2000000e+03, 3.2800000e+03, 1.1630000e+03, 1.4360000e+03,
       5.0000000e+03, 6.0000000e+03, 7.7500000e+03, 1.2380000e+03,
       6.5340000e+03, 6.0980000e+03, 6.9300000e+03, 2.9690000e+03,
       6.2500000e+03, 6.1030000e+03, 1.0250000e+03, 7.2500000e+03,
       1.2240000e+04, 1.1550000e+04, 6.8000000e+03, 6.6360000e+03,
       8.7750000e+03, 7.5000000e+03, 1.3678000e+04, 6.8750000e+03,
       9.7430000e+03, 7.0000000e+03, 7.0800000e+03, 5.9980000e+03,
       6.3000000e+03, 6.2230000e+03, 3.9440000e+03, 4.0946000e+04,
                 nan, 9.1660000e+03, 1.5830000e+03, 1.5890000e+03,
       8.5000000e+02, 5.2500000e+03, 1.0560000e+03, 6.0010000e+03,
       1.0010000e+03, 3.2000000e+03, 5.2000000e+03, 1.5116191e+07,
       6.6000000e+03, 6.1500000e+03, 1.1880000e+03, 4.5000000e+03,
       9.0500000e+03, 1.0100000e+03, 2.2956100e+05, 6.6070000e

In [ ]:
test = slo_re.head(10)
test

,MLS #,List Price,Sell Price,Address,Structure Type,Property Type,Beds,Full Baths,Half Baths,Square Footage,...,Closed On,Stories,Style,Attached Garage,Road Frontage,New Construction,Year Built,Common Interest,Days On Market,District
0,MLS #SC24127762,663500,663500,"570 Peach Street #23 San Luis Obispo, CA 93401...",House,Residential,2,2.0,0,918,...,2024-07-26,1,NaN,NaN,NaN,No,1975,Condominium,4.0,downtown
1,MLS #SC24091640,2195000,2000000,"570 Pacific San Luis Obispo, CA 93401",House,Residential,3,3.0,1,2227,...,2024-07-25,3,NaN,Yes,NaN,No,2024,NaN,57.0,downtown
2,MLS #SC24073717,1225000,1195000,"771 Toro Street San Luis Obispo, CA 93401-2830",House,Residential,2,2.0,0,1200,...,2024-07-19,1,NaN,NaN,NaN,No,1925,NaN,68.0,downtown
3,MLS #NS24109649,729000,729000,"1739 Johnson Avenue San Luis Obispo, CA 93401-...",House,Residential,2,1.0,0,900,...,2024-07-11,1,NaN,No,City Street,No,1925,NaN,8.0,downtown
4,MLS #SC23208408,1194000,1300000,"1143 Peach Street San Luis Obispo, CA 93401-2818",House,Residential,2,2.0,0,1100,...,2024-06-24,1,NaN,NaN,City Street,No,1906,NaN,143.0,downtown
5,MLS #SC24096310,999000,999000,"1052 Peach St San Luis Obispo, CA 93401",House,Residential,4,2.0,0,1564,...,2024-06-06,1,NaN,NaN,NaN,No,1926,NaN,3.0,downtown
6,MLS #NS24075035,765000,765000,"415 Dana Street #1 San Luis Obispo, CA 93401-3403",House,Residential,2,1.0,1,1463,...,2024-05-24,3,NaN,No,NaN,No,1983,Condominium,5.0,downtown
7,MLS #SC24061875,525000,525000,"680 Chorro Street #17 San Luis Obispo, CA 9340...",House,Residential,2,1.0,0,878,...,2024-05-09,2,Traditional,NaN,City Street,No,1980,Condominium,12.0,downtown
8,MLS #SC24047396,929000,929000,"1828 Johnson Avenue San Luis Obispo, CA 93401-...",House,Residential,4,1.0,0,1911,...,2024-05-07,2,NaN,NaN,NaN,No,1930,NaN,12.0,downtown
9,MLS #SC23118730,2395000,2395000,"576 -578 Pacific Street San Luis Obispo, CA 93401",House,Residential,4,4.0,1,2579,...,2024-05-03,3,"Contemporary, Modern",Yes,NaN,Yes,2023,NaN,NaN,downtown


In [ ]:
slo_re.shape[0]

157

In [ ]:
# Format new construction as binary
new_construction_mapping = {'Yes': 1, 'No': 0}

# Apply the mapping
slo_re['New Construction'] = slo_re['New Construction'].map(new_construction_mapping)

In [ ]:
# Feature engineer price difference variable

slo_re['Price Difference'] = slo_re['Sell Price'] / slo_re['List Price']
slo_re.head()

,MLS #,List Price,Sell Price,Address,Structure Type,Property Type,Beds,Full Baths,Half Baths,Square Footage,...,Stories,Style,Attached Garage,Road Frontage,New Construction,Year Built,Common Interest,Days On Market,District,Price Difference
0,MLS #SC24127762,663500,663500,"570 Peach Street #23 San Luis Obispo, CA 93401...",House,Residential,2,2.0,0,918,...,1,NaN,NaN,NaN,0,1975,Condominium,4.0,downtown,1.000000
1,MLS #SC24091640,2195000,2000000,"570 Pacific San Luis Obispo, CA 93401",House,Residential,3,3.0,1,2227,...,3,NaN,Yes,NaN,0,2024,NaN,57.0,downtown,0.911162
2,MLS #SC24073717,1225000,1195000,"771 Toro Street San Luis Obispo, CA 93401-2830",House,Residential,2,2.0,0,1200,...,1,NaN,NaN,NaN,0,1925,NaN,68.0,downtown,0.975510
3,MLS #NS24109649,729000,729000,"1739 Johnson Avenue San Luis Obispo, CA 93401-...",House,Residential,2,1.0,0,900,...,1,NaN,No,City Street,0,1925,NaN,8.0,downtown,1.000000
4,MLS #SC23208408,1194000,1300000,"1143 Peach Street San Luis Obispo, CA 93401-2818",House,Residential,2,2.0,0,1100,...,1,NaN,NaN,City Street,0,1906,NaN,143.0,downtown,1.088777


In [ ]:
geolocator = Nominatim(user_agent="input_email_address")

geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def clean_address(address):
    if pd.isna(address):
        return ""

    address = address.split(',')[0]


    address = re.sub(r'#\S+', '', address).strip()

    return address

def get_coordinates(cleaned_address):

    if cleaned_address.strip() == "":
        return None, None

    try:
        location = geocode(cleaned_address)
        if location:
            return location.latitude, location.longitude
        else:
            print(f"Geocoding failed for address: {cleaned_address}")
            return None, None
    except Exception as e:
        print(f"Error geocoding {cleaned_address}: {e}")
        return None, None

slo_re['Cleaned Address'] = slo_re['Address'].apply(clean_address)

slo_re[['Latitude', 'Longitude']] = slo_re['Cleaned Address'].apply(lambda addr: pd.Series(get_coordinates(addr)))

slo_re.head()

Geocoding failed for address: 706 Murray Avenue San Luis Obispo


Geocoding failed for address: 3872 Hayfield San Luis Obispo
Geocoding failed for address: 3935 Kilbern Way San Luis Obispo
Geocoding failed for address: 1252 Alder Court San Luis Obispo


Geocoding failed for address: 3909 Hayfield Loop San Luis Obispo
Geocoding failed for address: 1301 Bantam Court  San Luis Obispo


Geocoding failed for address: 157 Tango Way Lane San Luis Obispo
Geocoding failed for address: 153 Tango Way Lane San Luis Obispo


,MLS #,List Price,Sell Price,Address,Structure Type,Property Type,Beds,Full Baths,Half Baths,Square Footage,...,Road Frontage,New Construction,Year Built,Common Interest,Days On Market,District,Price Difference,Cleaned Address,Latitude,Longitude
0,MLS #SC24127762,663500,663500,"570 Peach Street #23 San Luis Obispo, CA 93401...",House,Residential,2,2.0,0,918,...,NaN,0,1975,Condominium,4.0,downtown,1.000000,570 Peach Street San Luis Obispo,35.281432,-120.667350
1,MLS #SC24091640,2195000,2000000,"570 Pacific San Luis Obispo, CA 93401",House,Residential,3,3.0,1,2227,...,NaN,0,2024,NaN,57.0,downtown,0.911162,570 Pacific San Luis Obispo,35.280136,-120.658902
2,MLS #SC24073717,1225000,1195000,"771 Toro Street San Luis Obispo, CA 93401-2830",House,Residential,2,2.0,0,1200,...,NaN,0,1925,NaN,68.0,downtown,0.975510,771 Toro Street San Luis Obispo,35.285476,-120.660686
3,MLS #NS24109649,729000,729000,"1739 Johnson Avenue San Luis Obispo, CA 93401-...",House,Residential,2,1.0,0,900,...,City Street,0,1925,NaN,8.0,downtown,1.000000,1739 Johnson Avenue San Luis Obispo,35.279243,-120.650931
4,MLS #SC23208408,1194000,1300000,"1143 Peach Street San Luis Obispo, CA 93401-2818",House,Residential,2,2.0,0,1100,...,City Street,0,1906,NaN,143.0,downtown,1.088777,1143 Peach Street San Luis Obispo,35.285429,-120.661198


In [ ]:
slo_re['Latitude'].unique().size

118

In [ ]:
slo_re['Price per/Sqft'] = slo_re['Sell Price'] / slo_re['Square Footage']
slo_re.head()

,MLS #,List Price,Sell Price,Address,Structure Type,Property Type,Beds,Full Baths,Half Baths,Square Footage,...,New Construction,Year Built,Common Interest,Days On Market,District,Price Difference,Cleaned Address,Latitude,Longitude,Price per/Sqft
0,MLS #SC24127762,663500,663500,"570 Peach Street #23 San Luis Obispo, CA 93401...",House,Residential,2,2.0,0,918,...,0,1975,Condominium,4.0,downtown,1.000000,570 Peach Street San Luis Obispo,35.281432,-120.667350,722.766885
1,MLS #SC24091640,2195000,2000000,"570 Pacific San Luis Obispo, CA 93401",House,Residential,3,3.0,1,2227,...,0,2024,NaN,57.0,downtown,0.911162,570 Pacific San Luis Obispo,35.280136,-120.658902,898.069151
2,MLS #SC24073717,1225000,1195000,"771 Toro Street San Luis Obispo, CA 93401-2830",House,Residential,2,2.0,0,1200,...,0,1925,NaN,68.0,downtown,0.975510,771 Toro Street San Luis Obispo,35.285476,-120.660686,995.833333
3,MLS #NS24109649,729000,729000,"1739 Johnson Avenue San Luis Obispo, CA 93401-...",House,Residential,2,1.0,0,900,...,0,1925,NaN,8.0,downtown,1.000000,1739 Johnson Avenue San Luis Obispo,35.279243,-120.650931,810.000000
4,MLS #SC23208408,1194000,1300000,"1143 Peach Street San Luis Obispo, CA 93401-2818",House,Residential,2,2.0,0,1100,...,0,1906,NaN,143.0,downtown,1.088777,1143 Peach Street San Luis Obispo,35.285429,-120.661198,1181.818182


In [ ]:
# Feature engineer baths
slo_re['Baths'] = slo_re['Full Baths'] + 0.5 * slo_re['Half Baths']

In [ ]:
slo_re.to_excel('SLO_RE_Clean.xlsx', index=False)